<img style="float: left;" src="http://sindser.org.br/s/wp-content/uploads/2013/09/iesb1.jpg"  width="400" height="400">

## Instituto de Educação Superior de Brasília
## Pós Graduação em Ciência de Dados
## Data Mining e Machine Learning II
## Victor Hugo - 1931133079


___

# <center>Tratamento e modelagem da base <a href='https://www.kaggle.com/ajay1735/hmeq-data'>hmeq_data</a></center>

# <a id='1'> 1. Dados</a>


## <a id= '1.1'> 1.1. Visão geral da base</a>
O dataset Home Equality (HMEQ), termo que se refere à diferença entre o preço de mercado da propriedade e o saldo pendente de todos os ônus atrelados a ela, contem uma base de informações sobre a performanse de 5960 emprestimos recentes voltados para a casa própria. A variável target (BAD) é do tipo binária e informa quando houve negligência no pagamento deste emprestimo. Esta realidade ocorreu em 1.890 dos casos (20%). Para cada indivíduo foram colhidas 12 variáveis.

# <a id='2'> 2. Exploração dos dados </a>

In [ ]:
# Importação das bibliotecas necessárias e leitura da base
import numpy as np # realização de calculos computacionais
import pandas as pd # manipulação dos dados
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')
df2 = df.copy()

## <a id='2.1'> 2.1. Variáveis </a>

* BAD (Realizou o pagamento do emprestimo "0", não realizou "1")
* LOAN (Quantidade de emprestimo solicitado)
* MORTDUE (Divida referente a uma hipotéca já existente)
* VALUE (Valor da propriedade atual)
* REASON (Motivo do emprestimo, sendo DebtCon = Consolidação de débitos e HomeImp = Melhorias na propriedade)
* JOB (Informa em qual das seis categorias ocupacionais está o indivíduo)
* YOJ (Quantidade de anos no atual emprego)
* DEROG (Quantidade de relatórios depreciativos, os principais)
* DELINQ (Quantidade de inadimplências em linhas de crédito) 
* CLAGE (Quantidade de meses desde a Trade Line mais antiga, trade line e responsável por gravar o comportamento de créditos ao consumidor)
* NINQ (Quantidade de linhas de crédito recentes)
* CLNO (Número total de linhas de Crédito)
* DEBTINC (Taxa de débitos que ainda estão por vir)

## <a id="2.2"> 2.2. Visualização da base </a>

In [ ]:
# Seleciona uma amostra com 10 observações da base e as mostra na tela
display(df.sample(10).T)

# Informa a quantidade de linhas e colunas respectivamente
display(df.shape)

In [ ]:
# Informações básicas da base como quantidade de colunas e seus respectivos nomes, quantidade des observações não nulas e seus respectivos tipos
df.info()

In [ ]:
def missing_values(data_frame):
    """
    Função responsável por mostrar quantos 
    valores missing há na base em cada 
    coluna
    """
    display(data_frame.isnull().sum().rename_axis('Colunas').reset_index(name='Missing Values'))


#chamada da função
missing_values(df)

In [ ]:
'''
seleciona as variáveis númericas da base para uma primeira análise, 
deixando de fora apenas a varíavel target "BAD", pois pelo fato da mesma 
ser binária, vizulizaremos-a melhor posteriormente em outro gráfico
'''
numeric_feats = [c for c in df.columns if df[c].dtype != 'object' and c not in ['BAD']]
df_numeric_feats = df[numeric_feats]


In [ ]:
# Cria um gráfico de paridade relacionado cada uma das variáveis entre si
sns.pairplot(df_numeric_feats)

Analisando estes gráficos gerados pode-se a uma primeira vista ver que há idícios de correlação entre a variável "VALUE" e "MORTDUE".

In [ ]:
# Cria histogramas das variáveis selecionadas anteriormente
df_numeric_feats.hist(figsize=(20,8), bins=30)
plt.tight_layout() 

Nestes gráficos percebemos que enquanto algumas variáveis estão mais próximos de uma distribuição normal outras se aproximam de uma distribuição poisson.

In [ ]:
'''
Cria uma sequência de gráficos relacionando 
as variáveis númericas anteriormente 
selecionadas com a variável target "BAD"
'''
plt.figure(figsize=(18,18))
c = 1
for i in df_numeric_feats.columns:
    if c < len(df_numeric_feats.columns):
        plt.subplot(3,3,c)
        sns.boxplot(x='BAD' , y= i, data=df)
        c+=1
    else:
        sns.boxplot(x='BAD' , y= i, data=df)
plt.tight_layout() 

Analisando estes gráficos, percebe-se que algumas das variáveis como "LOAN", "YOJ" e "CLAGE" apresentam uma média ligeiramente diferente entre os que pagaram e os que não pagaram os emprestimos tomados. E é possível notar também a quantidade de outliers presentes em cada uma dessas váriáveis, os quais podem estar consideravelmente influênciando o valor da média.

In [ ]:
# Cria um sequência de gráficos relacionando as varáveis númericas com a variável "JOB".
plt.figure(figsize=(18,18))
c = 1
for i in df_numeric_feats.columns:
    if c < len(df_numeric_feats.columns):
        plt.subplot(3,3,c)
        sns.boxplot(x='JOB' , y= i, data=df)
        c+=1
    else:
        sns.boxplot(x='JOB' , y= i, data=df)
plt.tight_layout() 

Assim como na plotagem anterior, nesta é possíve identificar as diferentes médias das váriaveis entre as diferentes ocupações do indivíduo que tomou o emprestimo. E também os outliers que podem estar consideravelmente influenciando no valor da média.

In [ ]:
#Dropa os campos com valores faltantes na coluna "JOB", sem alterar o datafram original, e pega os possíveis valores nesta coluna.
jobs = df['JOB'].dropna().unique()

#Cria uma série de histogramas da váriavel "VALUE" segmentando pela variável "JOB"
plt.figure(figsize=(14,15))
c=1
for i in jobs:
    plt.subplot(7,1,c)
    plt.title(i)
    df[df['JOB'] == i]['VALUE'].hist(bins=20)
    c+=1
plt.tight_layout() 


Como já demonstrado nos gráficos anteriores aqui consguimos perceber também a ligeira diferença entre as médias para esta variável para cada ocupaçã do indivíduo.

In [ ]:
# Motra um histograma dos valores presentes na variável "VALUE" onde na coluna JOB está faltando dado
print(df[df['JOB'].isnull()]['VALUE'].hist(bins=20))

In [ ]:
#Cria uma série de boxplot relacionando as variáveis númericas com a variável categórica "REASON".
plt.figure(figsize=(18,18))
c = 1
for i in df_numeric_feats.columns:
    if c < len(df_numeric_feats.columns):
        plt.subplot(3,3,c)
        sns.boxplot(x='REASON' , y= i, data=df)
        c+=1
    else:
        sns.boxplot(x='REASON' , y= i, data=df)


## <a id="2.3"> 2.3. Atestando algumas informações </a>

Para atestar estatisticamente algumas das informações obtidas com a visualização dos dados anteriormente será realizado alguns testes estatísticos.


In [ ]:
# Importando as bibliotecas necessárias
import scipy.stats as stats
from scipy.stats import ttest_ind

### <a id="2.3.1"> 2.3.1. Teste de Normalidade </a>

Primeiro, será realizado um teste para verificar a normalidade da distribuição dos valores da variável "VALUE" segmentada pela variável "REASON" para seguir então para o teste de diferença das médias. 

Para este teste será utilizado o teste de Shapiro–Wilk

$${{W} = {\frac{(\sum_{i=1}^{n} {a}_{i}{x}_{(i)})^{2}}{\sum_{i=1}^{n}({x}^{i} - \tilde{x})^{2}}}}$$

In [ ]:

# Seleciona os valores da variável "VALUE" onde o valor da varoável "REASON" e igual a "HomeImp"
df_reason_homeimp = df[df['REASON']=='HomeImp']['VALUE']
# Seleciona os valores da variável "VALUE" onde o valor da varoável "REASON" e igual a "DebtCon"
df_reason_debtcon = df[df['REASON']=='DebtCon']['VALUE']

# teste Shapiro-Wilk (Normalidade) para o subconjunto da variável "VALUE" onde o valor da variável "REASON" e igual a "HomeImp"
shapiro_stat_reason_homeimp, shapiro_p_valor_reason_homeimp = stats.shapiro(df_reason_homeimp)
# teste Shapiro-Wilk (Normalidade) para o subconjunto da variável "VALUE" onde o valor da variável "REASON" e igual a "DebtCon"
shapiro_stat_reason_debtcon, shapiro_p_valor_reason_debtcon = stats.shapiro(df_reason_debtcon)

# Mostra o P valor do teste de normalidade
print('teste de normalidade')
print('reason homeimp: {}'.format(shapiro_p_valor_reason_homeimp))
print('reason_debtcon: {}'.format(shapiro_p_valor_reason_debtcon))

In [ ]:
# Seleciona os valores da variável "VALUE" onde o valor da varoável "BAD" e igual a "1"
df_bad1 = df[df['BAD']== 1]['VALUE']
# Seleciona os valores da variável "VALUE" onde o valor da varoável "BAD" e igual a "0"
df_bad0 = df[df['BAD']== 0]['VALUE']

# teste Shapiro-Wilk (Normalidade) da biblioteca scipy para o subconjunto da variável "VALUE" onde o valor da variável "bAD" e igual a "1"
shapiro_stat_bad1,  shapiro_p_valor_bad1 = stats.shapiro(df_bad1)
# teste Shapiro-Wilk (Normalidade) da biblioteca scipy para o subconjunto da variável "VALUE" onde o valor da variável "bAD" e igual a "0"
shapiro_stat_bad0, shapiro_p_valor_bad0 = stats.shapiro(df_bad0)

# Mostra o P valor do teste de normalidade
print('teste de normalidade')
print('Value com Bad igual a 1: {}'.format(shapiro_p_valor_bad1))
print('Value com Bad igual a 0: {}'.format(shapiro_p_valor_bad0))

In [ ]:
# Seleciona os valores da variável "DEBTINC" onde o valor da varoável "BAD" e igual a "1"
df_debtinc_bad1 = df[df['BAD']== 1]['DEBTINC']
# Seleciona os valores da variável "DEBTINC" onde o valor da varoável "BAD" e igual a "0"
df_debtinc_bad0 = df[df['BAD']== 0]['DEBTINC']
shapiro_stat_bad1,  shapiro_p_valor_debtinc_bad1 = stats.shapiro(df_debtinc_bad1)
shapiro_stat_bad0, shapiro_p_valor_debtinc_bad0 = stats.shapiro(df_debtinc_bad0)

# Mostra o P valor do teste de normalidade
print('teste de normalidade')
print('Debtinc com Bad igual a 1: {}'.format(shapiro_p_valor_debtinc_bad1))
print('Debtinc com Bad igual a 0: {}'.format(shapiro_p_valor_debtinc_bad0))

Considerando um $\alpha $ = 0.05 não se discarta a hipotese nula, tendo em vista que o P Value resultado destes testes. Ou seja, estas distribuições se aproximam de uma distribuição normal.

### <a id="2.3.2"> 2.3.2. Teste de hipótese </a>

Considerando que a distribuiçãço destes dubconjuntos se aproxima de uma distribuição normal, a variáncia dos mesmos são parecidas e o tamanha das amostras são diferentes, será realizado o T-Test para testar a diferença entre as médias.

$${{t} = {\frac{\tilde{x}_{1} - \tilde{x}_{2}}{{S}_{x_1 x_2} .\sqrt{\frac{1}{n_1} + \frac{1}{n_2}}}}}$$

In [ ]:
# remove os valores campos com valores faltantes, sem alterar o dataframe original e realiza o t-test atraves da função ttest_ind da biblioteca scipy
_, ttest_p_value = ttest_ind(df_reason_homeimp.dropna(), df_reason_debtcon.dropna())

# Mostra o P valor  do t-teste
print('T-teste: {:.4f}'.format(ttest_p_value))

Considerando um $\alpha $ = 0.05, não se regeita a hipótese nula, sendo assim, as media dos valores da propriedades dos indivíduos que pegaram emprestimos para melhorias da propriedade (HomeImp) e a média do valor da propriedade dos indivíduos que pegaram emprestimos para a consolidação de créditos (DebtCon) não se diferem.

In [ ]:
# remove os valores campos com valores faltantes, sem alterar o dataframe original e realiza o t-test atraves da função ttest_ind da biblioteca scipy
_, ttest_p_value_bad = ttest_ind(df_bad1.dropna(), df_bad0.dropna())

# Mostr o P valor do teste
print('T-teste: {:.4f}'.format(ttest_p_value_bad))

Considerando um $\alpha $ = 0.05, regeita-se a hipótese nula, sendo assim, as media dos valores da propriedades dos indivíduos que não foram inadimplentes e os que foram, se diferem.

In [ ]:
# remove os valores campos com valores faltantes, sem alterar o dataframe original e realiza o t-test atraves da função ttest_ind da biblioteca scipy
_, ttest_p_value_debtinc_bad = ttest_ind(df_debtinc_bad1.dropna(), df_debtinc_bad0.dropna())

# Mostr o P valor do teste
print('T-teste: {:.4f}'.format(ttest_p_value_debtinc_bad))

Considerando um $\alpha $ = 0.05, regeita-se a hipótese nula, sendo assim, as media dos valores das taxas que ainda estão por vir dos indivíduos que estão em dia com seus emprestimos e a média do valores das taxas que ainda estão por vir dos indivíduos que fora inadimplentos em seus emprestimso se diferem.

### <a id='2.3.3'> 2.3.3. Análise de Variancia One Way </a>

Não é interessante utilizar os métodos para o teste de média entre duas variáveis como o T test para testar  diferença entre muitas muitas variáveis, pois isto se tornaria um processo muito oneroso. Para isso existe a analise de variâcia (Analise Of Variance - ANOVA)


In [ ]:
# pegando os valores da variável "VALUE" e agrupando por ocupação "JOB"
anova_value_by_job = {job:df['VALUE'][df['JOB'] == job] for job in jobs}

# realizando o teste de análise de variácia
_, anova_value_job_p = stats.f_oneway(anova_value_by_job['Other'].dropna(),
                                          anova_value_by_job['Office'].dropna(),
                                          anova_value_by_job['Sales'].dropna(),
                                          anova_value_by_job['Mgr'].dropna(),
                                          anova_value_by_job['ProfExe'].dropna(), 
                                          anova_value_by_job['Self'].dropna())
# Mostra o P value do teste
print('One Way Anova: {:.4f}'.format(anova_value_job_p))

Considerando um $\alpha $ = 0.05, podemos dizer que a média de "VALUE" em pelo menos uma das ocupações e diferente das demais.

No intúito de verificar se removendo os outliers estas médias continuariam resultando em um P valor baixo para a ANOVA, fora realizada a remoção dos outliers utilizando a regra do IQR (Inter Quartile Range) que diz que valores menores que 1.5 * Quartil 1 ou maiores que 1.5 * o Quartil 3 são considerados outliers.

In [ ]:
# Calculando o primeiro quartil da variável "VALUE"
q1 = df['VALUE'].quantile(0.25)
# Calculando o terceiro quartil da variável "VALUE"
q3 = df['VALUE'].quantile(0.75)

# Calculando o IQR
iqr = q3 - q1 

# Guardando domente os valores que não são considerados outliers 
df_value_and_job_no_outlier = df[~((df['VALUE'] < (q1 - 1.5  * iqr)) | (df['VALUE']  > (q3 + 1.5 * iqr)))][['VALUE', 'JOB', 'BAD']]

In [ ]:
# Verificando se as medias continuao diferentes
anova_value_by_job = {job:df_value_and_job_no_outlier['VALUE'][df_value_and_job_no_outlier['JOB'] == job] for job in jobs}
anova_job_f, anova_job_p = stats.f_oneway(anova_value_by_job['Other'].dropna(),
                                          anova_value_by_job['Office'].dropna(),
                                          anova_value_by_job['Sales'].dropna(),
                                          anova_value_by_job['Mgr'].dropna(),
                                          anova_value_by_job['ProfExe'].dropna(), 
                                          anova_value_by_job['Self'].dropna())
#Mostra o P valor do teste
print('One Way Anova: {:.4f}'.format(anova_job_p))

Houve mundança no resultado, contudo, ainda assim, considerando o alpha já estabelecido de 5%, a média do valor da variável "VALUE" em pelo menos uma das ocupações e diferente das demais.

In [ ]:
sns.boxplot(x='JOB', y='VALUE', data=df_value_and_job_no_outlier)

In [ ]:

anova_debtinc_by_job = {job:df['DEBTINC'][df['JOB'] == job] for job in jobs}
anova_debtinc_f, anova_debtinc_p = stats.f_oneway(anova_debtinc_by_job['Other'].dropna(),
                                          anova_debtinc_by_job['Office'].dropna(),
                                          anova_debtinc_by_job['Sales'].dropna(),
                                          anova_debtinc_by_job['Mgr'].dropna(),
                                          anova_debtinc_by_job['ProfExe'].dropna(), 
                                          anova_debtinc_by_job['Self'].dropna())
#Ao menos um dos Jobs tem valores diferentes entre si, estatisticamente falando
print('One Way Anova: {:.4f}'.format(anova_debtinc_p))

Verificando agora a variável "DEBTINC" pela ocupação dos indivíduos, considerando o P valor ao menos um das ocupações apresenta um valor médio diferente das demais para a variável "DEBTINC".

Para a variável "YOJ" será realizado a mesma verificação retirando os outliers.

In [ ]:
# Selecionando o primeiro Quartil da variável "YOJ"
q1 = df['YOJ'].quantile(0.25)
# Selecionando o segundo Quartil da variável "YOJ"
q3 = df['YOJ'].quantile(0.75)

#Ralizando o calculo do iqr
iqr = q3 - q1

#descartando os outliers e 
df_yoj_and_job_no_outlier = df[~((df['YOJ'] < (q1 - 1.5  * iqr)) | (df['YOJ']  > (q3 + 1.5 * iqr)))][['YOJ', 'JOB']]

In [ ]:

anova_yoj_by_job = {job:df_yoj_and_job_no_outlier['YOJ'][df_yoj_and_job_no_outlier['JOB'] == job] for job in jobs}
anova_yoj_f, anova_yoj_p = stats.f_oneway(anova_yoj_by_job['Other'].dropna(),
                                          anova_yoj_by_job['Office'].dropna(),
                                          anova_yoj_by_job['Sales'].dropna(),
                                          anova_yoj_by_job['Mgr'].dropna(),
                                          anova_yoj_by_job['ProfExe'].dropna(), 
                                          anova_yoj_by_job['Self'].dropna())
#Ao menos um dos Jobs tem valores diferentes entre si, estatisticamente falando
print('One Way Anova: {:.4f}'.format(anova_yoj_p))

Assim como as anteriores e conforme mostrado nos gáficos, estatisticamente há pelo menos uma das categorias que possui o valor médio na variável "YOJ" diferente das demais.

In [ ]:
sns.boxplot(x='JOB', y= 'YOJ', data=df_yoj_and_job_no_outlier)

# <a id='3'> 3. Manipulação da base </a>

## <a id='3.1'> 3.1. Imputação dos dados </a>

Agora que já foram atestadas as informações que fora percebido na analise dos gráficos, estas serão utilizadas para o tratamento da base.
O valor médio da variável "VALUE" foi constatado que difera entre os diferentes tipos de ocupação do indivíduo, variável "JOB", e difere também entre os indivíduos adimplentes e inadimlentes, sendo assim um metodo conciso para para a imputação dos dados seria pela média de "VALUE" por "JOB" e BAD. 

In [ ]:
# Salvando as médias da variável VALUE por ocupação
value_mean_by_job = df_value_and_job_no_outlier.groupby(['JOB', 'BAD'])['VALUE'].mean()

# instancia um objeto pandas series sem conteúdo.
imp_value = pd.Series([]) 

'''
reseta o idice do data frame para garantir 
que cada iteração verifique um ídice do 
dataframe evitando com que observações 
fiquem sem ser verificadas.
'''
df.reset_index()
'''
itera sobre o dataframe e, caso valor do 
campo "VALUE" esteja nulo, verifica a 
ocupação do indivíduo e coloca a média de 
"VALUE" para aquele "JOB" naquela posição 
dentro de um objeto Series, caso "VALUE"
não seja nulo, atribui ao objeto o próprio
valor de "VALUE"
'''
for i in range(len(df)):
    if df['VALUE'][i] != df['VALUE'][i]:
        if df['JOB'][i] == 'Mgr':
            if df['BAD'][i] == 0:
                imp_debtinc[i] = value_mean_by_job['Mgr'][0]
            else:
                imp_value[i] = value_mean_by_job['Mgr'][1]
        if df['JOB'][i] == 'Office':
            if df['BAD'][i] == 0:
                imp_value[i] = value_mean_by_job['Office'][0]
            else:
                imp_value[i] = value_mean_by_job['Office'][1]
        if df['JOB'][i] == 'Other':
            if df['BAD'][i] == 0:
                imp_value[i] = value_mean_by_job['Other'][0]
            else:
                imp_value[i] = value_mean_by_job['Other'][1]
        if df['JOB'][i] == 'ProfExe':
            if df['BAD'][i] == 0:
                imp_value[i] = value_mean_by_job['ProfExe'][0]
            else:
                imp_value[i] = value_mean_by_job['ProfExe'][1]
        if df['JOB'][i] == 'Sales':
            if df['BAD'][i] == 0:
                imp_value[i] = value_mean_by_job['Sales'][0]
            else:
                imp_value[i] = value_mean_by_job['Sales'][1]
        if df['JOB'][i] == 'Self':
            if df['BAD'][i] == 0:
                imp_value[i] = value_mean_by_job['Self'][0]
            else:
                imp_value[i] = value_mean_by_job['Self'][1]
            
    else: 
        imp_value[i] = df['VALUE'][i]
'''
casi já exista alguma coluna com o nome IMP_VALUE
realiza a exclusão o mesmo
'''
if "IMP_VALUE" in np.array(df.columns):
    df.drop("IMP_VALUE", axis=1, inplace=True)
    
# Inserie o objeto no dataframe como uma coluna
df.insert(13, "IMP_VALUE", imp_value) 
df.head().T

In [ ]:
# Seleciona as observações do dataframe onde a coluna "IMP_VALUE" apresenta valores faltantes
df[df['IMP_VALUE'].isnull()]

Vemos que alguns campos ainda ficaram com dados faltantes, pois o campo "JOB" também possui dados faltantes. Ao mesmo tempo é possíve ver que existem algumas obsesrvações que preticamente só possuem o valor da variável "BAD" e "LOAN", neste caso a melhor opção no momento foi dropar esta observações que possuem mais que 10 campos missings.

In [ ]:
# Descarta todas as observações que possuam mais que 10 campos com valores faltantes.
df.dropna(thresh=10, inplace=True)
# Mostra a estrutura do dataframe
df.shape

In [ ]:
missing_values(df)

In [ ]:
# Seleciona as observações do dataframe onde a coluna "IMP_VALUE" apresenta valores faltantes
df[df['IMP_VALUE'].isnull()]

In [ ]:
df.dropna(axis=0,subset=['IMP_VALUE'], inplace=True)
df.shape

Agora com a variável "IMP_VALUE", a variável "VALUE" não será mais necessária, tendo em vista que transmite a mesma informação. Sendo assim, a mesma será descartada.

In [ ]:
df.drop('VALUE', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
missing_values(df)

Dentro das variáveis com valores faltantes, temos as duas variáveis qualititavas da base "JOB" e "REASON".
Tendo em vista as diferentes médias que as outras variáveis tem quando agrupadas por estas variáveis, seria interessante tentar imputar esta coluna utilizando um ensemble metodo como GBM ou Random Forest, contudo, por hora, foi decidido apenas dropar estas observações.

In [ ]:
df.dropna(axis=0, subset=['JOB'], inplace=True)
df.dropna(axis=0, subset=['REASON'], inplace=True)

In [ ]:
df.shape

In [ ]:
missing_values(df)

Anteriormente durante a análise gráfico percebeu-se uma certa correlação entre a variável "VALUE" agora "IMP_VALUE" e a variável "MORTDUE".

In [ ]:
df[['IMP_VALUE', 'MORTDUE']].corr()

In [ ]:
plt.scatter(df['IMP_VALUE'], df['MORTDUE'])
plt.ylabel('IMP_VALUE')
plt.xlabel('MORTDUE')
plt.show()

Sendo assim, uma forma interessante de se imputar os dados neste caso seria através de uma regressão linear.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
missing_mortdue = df[df['MORTDUE'].isnull()][['IMP_VALUE', 'MORTDUE']]
not_missing_mortdue = df[df['MORTDUE'].notnull()][['IMP_VALUE', 'MORTDUE']]

In [ ]:
X = not_missing_mortdue['IMP_VALUE'].values.reshape(-1, 1)
y = not_missing_mortdue['MORTDUE'].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
mortdue_pred = lr.predict(X_test)

In [ ]:
real_vs_pred = pd.DataFrame({'Real': y_test.flatten(), 'Predito': mortdue_pred.flatten()})
real_vs_pred.sample(20)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(X_test, y_test, color='gray')
plt.plot(X_test, mortdue_pred, color='red', linewidth=2)
plt.show()

In [ ]:
print('Raiz quadrada do Erro medio ao quadrado: {}'.format(np.sqrt(metrics.mean_squared_error(y_test, mortdue_pred))))
print('R quadrado: {}'.format(metrics.r2_score(y_test, mortdue_pred)))

In [ ]:
# trantando outliers para tentar diminuir o erro.
# calculando o iqr
q1 = not_missing_mortdue.quantile(0.25)
q3 = not_missing_mortdue.quantile(0.75)

iqr = q3-q1

print(iqr)
not_missing_and_outliers_mortdue = not_missing_mortdue[~((not_missing_mortdue < (q1 - 1.5  * iqr)) | (not_missing_mortdue > (q3 + 1.5 * iqr))).any(axis=1)]

In [ ]:
not_missing_mortdue.shape

In [ ]:
not_missing_and_outliers_mortdue.shape

In [ ]:
X_no_outliers = not_missing_and_outliers_mortdue['IMP_VALUE'].values.reshape(-1, 1)
y_no_outliers = not_missing_and_outliers_mortdue['MORTDUE'].values.reshape(-1, 1)
X_no_outliers_train, X_no_outliers_test, y_no_outliers_train, y_no_outliers_test = train_test_split(X_no_outliers, y_no_outliers, test_size=0.20, random_state=42)

In [ ]:
lr_no_outliers = LinearRegression()
lr_no_outliers.fit(X_no_outliers_train, y_no_outliers_train)

In [ ]:
mortdue_pred = lr.predict(X_test)
real_vs_pred = pd.DataFrame({'Real': y_test.flatten(), 'Predito': mortdue_pred.flatten()})
real_vs_pred.sample(20)

In [ ]:
print('Raiz quadrada do Erro medio ao quadrado: {:.2f}'.format(np.sqrt(metrics.mean_squared_error(y_test, mortdue_pred))))
print('R quadrado: {}'.format(metrics.r2_score(y_test, mortdue_pred)))

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(X_test, y_test, color='gray')
plt.plot(X_test, mortdue_pred, color='red', linewidth=2)
plt.show()

In [ ]:
imp_mortdue = pd.Series([])
imp_mortdue = lr.predict(df['IMP_VALUE'].values.reshape(-1,1))
imp_mortdue

In [ ]:
df.insert(13, 'IMP_MORTDUE', np.round(imp_mortdue, 2))
df.drop('MORTDUE', axis=1, inplace=True)


In [ ]:
df.head()

In [ ]:
missing_values(df)

In [ ]:
sns.boxplot(x='JOB', y='DEBTINC', data=df)

In [ ]:
sns.boxplot(x='BAD', y='DEBTINC', data=df)

In [ ]:
q1 = df['DEBTINC'].quantile(0.25)
q3 = df['DEBTINC'].quantile(0.75)

iqr = q3 - q1

df_debtinc_and_job_no_outlier = df[~((df['DEBTINC'] < (q1 - 1.5  * iqr)) | (df['DEBTINC']  > (q3 + 1.5 * iqr)))][['DEBTINC', 'JOB', 'BAD']]
debtinc_mean_by_job = df_debtinc_and_job_no_outlier.groupby(['JOB', 'BAD'])['DEBTINC'].mean()
debtinc_mean_by_job


In [ ]:
debtinc_mean_by_job['Mgr'][1]

In [ ]:
imp_debtinc = pd.Series([]) 

df.reset_index(inplace=True)
for i in range(len(df)):
    if df['DEBTINC'][i] != df['DEBTINC'][i]:
        if df['JOB'][i] == 'Mgr':
            if df['BAD'][i] == 0:
                imp_debtinc[i] =  debtinc_mean_by_job['Mgr'][0]
            else:
                imp_debtinc[i] =  debtinc_mean_by_job['Mgr'][1]
        if df['JOB'][i] == 'Office':
            if df['BAD'][i] == 0:
                imp_debtinc[i] =  debtinc_mean_by_job['Office'][0]
            else:
                imp_debtinc[i] =  debtinc_mean_by_job['Office'][1]
        if df['JOB'][i] == 'Other':
            if df['BAD'][i] == 0:
                imp_debtinc[i] =  debtinc_mean_by_job['Other'][0]
            else:
                imp_debtinc[i] =  debtinc_mean_by_job['Other'][1]
        if df['JOB'][i] == 'ProfExe':
            if df['BAD'][i] == 0:
                imp_debtinc[i] =  debtinc_mean_by_job['ProfExe'][0]
            else:
                imp_debtinc[i] =  debtinc_mean_by_job['ProfExe'][1]
        if df['JOB'][i] == 'Sales':
            if df['BAD'][i] == 0:
                imp_debtinc[i] =  debtinc_mean_by_job['Sales'][0]
            else:
                imp_debtinc[i] =  debtinc_mean_by_job['Sales'][1]
        if df['JOB'][i] == 'Self':
            if df['BAD'][i] == 0:
                imp_debtinc[i] =  debtinc_mean_by_job['Self'][0]
            else:
                imp_debtinc[i] =  debtinc_mean_by_job['Self'][1]
    else: 
        imp_debtinc[i] = df['DEBTINC'][i]

if "IMP_DEBTINC" in np.array(df.columns):
    df.drop("IMP_DEBTINC", axis=1, inplace=True)
    
df.insert(13, "IMP_DEBTINC", imp_debtinc) 
df.head().T

In [ ]:
df.shape

In [ ]:
missing_values(df)

In [ ]:
df.drop('DEBTINC', axis=1, inplace=True)

In [ ]:
sns.boxplot(x='JOB', y='YOJ', data=df)

In [ ]:
yoj_mean_by_job = df_yoj_and_job_no_outlier.groupby(['JOB'])['YOJ'].mean()
imp_yoj = pd.Series([]) 

df.reset_index(inplace=True)
for i in range(len(df)):
    if df['YOJ'][i] != df['YOJ'][i]:
        if df['JOB'][i] == 'Mgr':
            imp_yoj[i] =  yoj_mean_by_job['Mgr']
        if df['JOB'][i] == 'Office':
            imp_yoj[i] = yoj_mean_by_job['Office']
        if df['JOB'][i] == 'Other':
            imp_yoj[i] = yoj_mean_by_job['Other']
        if df['JOB'][i] == 'ProfExe':
            imp_yoj[i] = yoj_mean_by_job['ProfExe']
        if df['JOB'][i] == 'Sales':
            imp_yoj[i] = yoj_mean_by_job['Sales']
        if df['JOB'][i] == 'Self':
            imp_yoj[i] = yoj_mean_by_job['Self']
    else: 
        imp_yoj[i] = df['YOJ'][i]
        
if "IMP_YOJ" in np.array(df.columns):
    df.drop("IMP_YOJ", axis=1, inplace=True)
    
df.insert(13, "IMP_YOJ", imp_yoj) 
df.head().T

In [ ]:
df.drop('YOJ', axis=1, inplace=True)

In [ ]:
missing_values(df)

In [ ]:
# Em virtude a falta de tempo para as outra variaveis (DEROG,DELINQ, CLAGE e NINQ)  fora imputadas pela media 
df.fillna(df.mean(), inplace=True)

In [ ]:
missing_values(df)

In [ ]:
# antes de ir para o modelo propriamente dito vou realizar um one hot encoding nas variaveis categoricas.
df = pd.get_dummies(data=df, columns=['JOB', 'REASON'])

In [ ]:
df.head()

In [ ]:
# Iniciando o modelo de predicao

In [ ]:
#pip install -U scikit-learn == 0.22.1


In [ ]:
#pip install -U imbalanced-learn

In [ ]:
# Classes desbalanceadas, vamos rodar um modelo com as classes do jeito que estao.
df['BAD'].value_counts().plot(kind='bar')

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.metrics import classification_report


In [ ]:
feats = [c for c in df.columns if c not in ['BAD']]

In [ ]:
X = df[feats]
y = df['BAD']

In [ ]:
param_grid_decision_tree = {
    'criterion': ('gini', 'entropy'),
    'splitter': ('best', 'random'),
    'max_features': ('auto', 'sqrt', 'log2')
}

In [ ]:
grid_decision_tree = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid_decision_tree)

In [ ]:
grid_decision_tree.fit(X,y)

In [ ]:
print('\nOs melhores parametros foram: \n' + str(grid_decision_tree.best_params_))

In [ ]:
cv_decision_tree = cross_val_score(grid_decision_tree, X, y, cv=10)
print(cv_decision_tree)
print(cv_decision_tree.mean())

In [ ]:
param_grid_random_forest = {
    'criterion': ('gini', 'entropy'),
    'max_features': ('log2', 'sqrt')
}


In [ ]:
grid_random_forest_classifier = GridSearchCV(RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1, bootstrap = True, oob_score = True), param_grid_random_forest)
grid_random_forest_classifier.fit(X,y)

In [ ]:
print('\nOs melhores parametros foram: \n' + str(grid_random_forest_classifier.best_params_))

In [ ]:
cv_random_forest = cross_val_score(grid_random_forest_classifier, X, y, cv=10, n_jobs=-1)
print(cv_random_forest)
print(cv_random_forest.mean())

In [ ]:
param_grid_gbost_classifier = {
    'criterion': ['friedman_mse', 'mse', 'mae'],
    'max_features': ('log2', 'sqrt')
} 

In [ ]:
grid_gradiente_boost_machine = GridSearchCV(GradientBoostingClassifier(n_estimators=200, learning_rate=1.0, max_depth=1, random_state=42, n_jobs=-1), param_grid_gbost_classifier)
grid_gradiente_boost_machine.fit(X, y)

In [ ]:
print('\nOs melhores parametros foram: \n' + str(grid_gradiente_boost_machine.best_params_))

In [ ]:
cv_gradiente_boost_machine = cross_val_score(grid_gradiente_boost_machine, X, y, cv=10)
print(cv_gradiente_boost_machine)
print(cv_gradiente_boost_machine.mean())

In [ ]:
# realizando um over sapling da classe minoritaria
smt = SMOTE(sampling_strategy=0.80)
X, y = smt.fit_sample(X,y)

In [ ]:
y.value_counts().plot(kind='bar')

In [ ]:
grid_decision_tree.fit(X, y)

In [ ]:
cv_decision_tree = cross_val_score(grid_decision_tree, X, y, cv=10, n_jobs=-1)
print(cv_decision_tree)
print(cv_decision_tree.mean())

In [ ]:
grid_random_forest_classifier.fit(X, y)

In [ ]:
cv_random_forest = cross_val_score(grid_random_forest_classifier, X, y, cv=10, n_jobs=-1)
print(cv_random_forest)
print(cv_random_forest.mean())

In [ ]:
grid_gradiente_boost_machine.fit(X, y)

In [ ]:
cv_gradiente_boost_machine = cross_val_score(grid_gradiente_boost_machine, X, y, cv=10, n_jobs=-1)
print(cv_gradiente_boost_machine)
print(cv_gradiente_boost_machine.mean())